In [ ]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
tf.__version__  #must be 1.15

In [ ]:

datadir = 'Data/' # path to the data directory

In [ ]:
import glob

def read_mm(mfile):
    hm=np.memmap(mfile, dtype=np.int32, shape=(4,))
    shape = tuple(hm)
    del hm

    mm = np.memmap(mfile, dtype=np.float32, shape=shape)
    mm.shape
    
    amm = mm[:,:,:,[2, 0, 4, 0, 1]]
    amm = amm[:,:,:, [0,2,4]]
    del mm
    return amm

def read_data(globpattern):
    ifiles = glob.glob(globpattern)
    ammdict = dict()
    for i in ifiles:
        amm = read_mm(i)
        amm_m = amm.mean(axis=(1,2), keepdims=True)
        amm_std = amm.std(axis=(1,2), keepdims=True)
        amm = (amm - amm_m)/amm_std
        print(amm.shape)
        ammdict[i.split('/')[-1]] = amm
   
    return ammdict

ammdict = read_data('Data/path_to_data/*.mm') 

In [ ]:
tf.reset_default_graph()
sess = tf.Session()


In [ ]:
cpdir = ''  # Checkpoint directory
checkpoint = ''  #checkpoint name
if not cpdir.endswith('/'):
    cpdir += '/'

cpmeta = cpdir + checkpoint + '.meta'
saver = tf.train.import_meta_graph(cpmeta)
saver.restore(sess, cpdir + checkpoint)

In [ ]:
pq = 'dense_1/BiasAdd'.lower()
print(pq)
g = tf.get_default_graph()
ops = g.get_operations()
for op in ops:
    if pq in op.name.lower():
        print(op.name, op.outputs)

In [ ]:
'''
a : the layer in the network before the last relu activation
b : the last relu activation, just before softmax
sm : the softmax probabilities
batch : the placeholder for input images
'''
a = sess.graph.get_tensor_by_name('dense_1/BiasAdd:0')
b = sess.graph.get_tensor_by_name('LeakyRelu_13:0') #'dense_1/BiasAdd:0')
sm = sess.graph.get_tensor_by_name('Softmax:0')

softmax = sess.graph.get_tensor_by_name('Softmax:0')
batch = sess.graph.get_tensor_by_name('Placeholder:0')

In [ ]:
a, b, sm

In [ ]:
ammdict[list(ammdict.keys())[2]].shape

In [ ]:
'''
This is the cell where the inferring happens.

use sess.run to get the last three layers of the network
the softmax prediction (array of number of classes) is p2
the other two layers are d2 and r2

the dataframe will be
sm : the predicted class
smmax : the probability of the predicted class
dataset : dataset the image is in (from input file)
'''

images = sess.graph.get_tensor_by_name('Placeholder:0')
adflist = list()
d2list = list()
for k, v in ammdict.items():
    d2, r2, p2 = sess.run([a, b, sm], feed_dict={images:v[:, 16:48, 16:48, :]})
    d2list.append(d2)
    p2x = np.argmax(p2, axis=-1)
    s = k.split('_')
    dataset = k
    xdf = pd.DataFrame({'sm':p2x, 'smmax':p2.max(axis=-1), 'dataset':dataset})
    adflist.append(xdf)

adf = pd.concat(adflist, axis=0)
d2array = np.concatenate(d2list)
d2array.shape

In [ ]:
adf.head()


In [ ]:
radf = adf.reset_index()

In [ ]:
radf.to_pickle("")  # save the dataframe to a pickle to analyze later